### Clipping

En un juego a menudo tenemos la necesidad de dibujar solo en una pate 
de la pantalla. Por ejemplo, en un juego de estrategia, al estilo
del [Command and Conquer](https://es.wikipedia.org/wiki/Command_%26_Conquer), podemos
querer dividir la pantalla en dos. Una parte superior grande donse se muestra un mapa
y un panel inferior, más pequeño, donde se muestre información de nuestras tropas,
del estado de municiones, etc... Obviamente, no queremos que al dibujar el mapa
pinte cosas enciam del panel de estado, ni al contrario. 

Para solucionar esto se puede usar una característica de las superficies de pygame
que es al _clipping area_ o área de recorte. Cada superficie tiene una área de recorte, 
un rectángulo (`un objeto de la clase `pygame.Rect`) que puede estar activa a no.  Si 
definimos y activamos el area, todas las operaciones de
dibujo se verán limitadas al rectángulo del área, dejando el resto de la superficie 
intacta.

Para definir un area de recorte,se usa el método `set_clip` de las superficies, pasandole
como parámetro un objeto de tipo `Rect`. Se puede obtener el aárea definida en cualquier
momento llamanda a `get_clip`. 

El siguiente codigo muestra una simulacion de un juego de estrategia con la configuración 
explicada antes. La primera llamada a `set_clip` define como área sobre la que se puede pintar
la parte superior, correspondiente al mapa (definido en el rectángulo `map_area`, y la
segunda limita el área utilizable a el area de información:

Often when you are building a screen for a game, you will want to draw only to a portion of the
display. For instance, in a strategy Command & Conquer–like game, you might have the top of
the screen as a scrollable map, and below it a panel that displays troop information. But when
you start to draw the troop images to the screen, you don’t want to have them draw over the
information panel. To solve this problem, surfaces have a clipping area, which is a rectangle
that defines what part of the screen can be drawn to. To set the clipping area, call the set_clip
method of a surface object with a Rect-style object. You can also retrieve the current clipping
region by calling get_clip.
The following snippet shows how we might use clipping to construct the screen for a strat-
egy game. The first call to clip sets the region so that the call to draw_map will only be able to
draw onto the top half of the screen. The second call to set_clip sets the clipping area to the
remaining portion of the screen:
screen.set_clip(0, 0, 640, 300)
draw_map()
screen.set_clip(0, 300, 640, 180)
draw_panel()

In [18]:
import pygame
import random

SIZE = WIDTH, HEIGHT = 800, 600
FPS = 60
BLACK = (0, 0, 0)
GRAY = (128, 128, 128)
CYAN = (0, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)

class Soldier:
    
    def __init__(self, x=None, y=None):
        self.x = x or random.randrange(WIDTH)
        self.y = y or random.randrange(HEIGHT)
        
    def update(self):
        self.x += random.choice([-2, -1, -1, 0, 0, 0, 1, 1, 2])
        self.y += random.choice([-2, -1, -1, 0, 0, 0, 1, 1, 2])
        
class Troop(Soldier):
    
    def draw(self, canvas):
        pygame.draw.circle(canvas, CYAN, (self.x, self.y), 5)
        
    def is_enemy(self):
        return False

class Enemy(Soldier):
    
    def draw(self, canvas):
        r = pygame.Rect(self.x-5, self.y-5, 11, 11)
        pygame.draw.rect(canvas, RED, r)
        
    def is_enemy(self):
        return True
        
map_area = pygame.Rect((0, 0), (WIDTH, HEIGHT-40))
info_area = pygame.Rect((0, HEIGHT-32), (WIDTH, 40))
        
pygame.init()
try:
    pygame.display.set_caption("Clippeng Demo")
    screen = pygame.display.set_mode(SIZE, 0, 24)

    # Parte de inicialización del juego
    crowd = [Troop() for _ in range(10)] + [Enemy() for _ in range(12)]
    clock = pygame.time.Clock()    
    in_game = True

    while in_game:
        # Obtener datos de entrada
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                in_game = False
        # Recalcular el estado del juego, en base al estado actual y a las entradas
        for soldier in crowd:
            soldier.update()
        # Representamos el nuevo estado
        screen.set_clip(map_area)
        screen.fill(BLACK)
        for soldier in crowd:
            soldier.draw(screen)
        screen.set_clip(info_area)
        screen.fill(GRAY)
        pos = 5
        for soldier in crowd:            
            r = pygame.Rect((pos, 0), (4, HEIGHT))
            if soldier.is_enemy():
                pygame.draw.rect(screen, RED, r)
            else:
                pygame.draw.rect(screen, GREEN, r)
            pos += 5
        pygame.display.update()
        clock.tick(FPS)
finally:
    pygame.quit()

**Ejercicio**: Modificar el programa para que detecte los eventos de pulsar 
la tecla del ratón. Si en las coordenadas pulsadas hay un soldado (Ya 
sea nuestro o del enemigo), borrarlo. Para nuestros efectos, borrarlo es
simplemente eliminarlo de la lista `crowd`. Para eso se puede usar 
el método `remove(elem)` de las listas.

In [21]:
l = [1, 2, 3, 4, 5, 6]
print(l)
l.remove(4)
print(l)

[1, 2, 3, 4, 5, 6]
[1, 2, 3, 5, 6]
